<a href="https://colab.research.google.com/github/Meta-Sean/Course-6-Mono/blob/main/6_0002_ps2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# 6.0002 Problem Set 5
# Graph optimization
# Name:
# Collaborators:
# Time:

import unittest

#
# A set of data structures to represent graphs
#

class Node(object):
    """Represents a node in the graph"""
    def __init__(self, name):
        self.name = str(name)

    def get_name(self):
        return self.name

    def __str__(self):
        return self.name

    def __repr__(self):
        return self.name

    def __eq__(self, other):
        return self.name == other.name

    def __ne__(self, other):
        return not self.__eq__(other)

    def __hash__(self):
        # This function is necessary so that Nodes can be used as
        # keys in a dictionary, even though Nodes are mutable
        return self.name.__hash__()


class Edge(object):
    """Represents an edge in the dictionary. Includes a source and
    a destination."""
    def __init__(self, src, dest):
        self.src = src
        self.dest = dest

    def get_source(self):
        return self.src

    def get_destination(self):
        return self.dest

    def __str__(self):
        return '{}->{}'.format(self.src, self.dest)


class WeightedEdge(Edge):
    def __init__(self, src, dest, total_distance, outdoor_distance):
        self.src = src
        self.dest = dest
        self.total_distance = total_distance
        self.outdoor_distance = outdoor_distance

    def get_total_distance(self):
        return self.total_distance

    def get_outdoor_distance(self):
        return self.outdoor_distance

    def __str__(self):
        return f"{self.src}->{self.dest} ({self.total_distance}, {self.outdoor_distance})"


class Digraph(object):
    """Represents a directed graph of Node and Edge objects"""
    def __init__(self):
        self.nodes = set([])
        self.edges = {}  # must be a dict of Node -> list of edges

    def __str__(self):
        edge_strs = []
        for edges in self.edges.values():
            for edge in edges:
                edge_strs.append(str(edge))
        edge_strs = sorted(edge_strs)  # sort alphabetically
        return '\n'.join(edge_strs)  # concat edge_strs with "\n"s between them

    def get_edges_for_node(self, node):
        return self.edges[node]

    def has_node(self, node):
        return node in self.nodes

    def add_node(self, node):
        """Adds a Node object to the Digraph. Raises a ValueError if it is
        already in the graph."""
        if node in self.edges:
          raise ValueError('Duplicate node')
        else:
          self.edges[node] = []

    def add_edge(self, edge):
        """Adds an Edge or WeightedEdge instance to the Digraph. Raises a
        ValueError if either of the nodes associated with the edge is not
        in the  graph."""
        src = edge.get_source()
        dest = edge.get_destination()
        total = edge.get_total_distance()
        outdoor = edge.get_outdoor_distance()
        we = WeightedEdge(src, dest, total, outdoor)
        if not (src in self.edges and dest in self.edges):
          raise ValueError("Node not in graph")
        self.edges[src].append(we)


# ================================================================
# Begin tests -- you do not need to modify anything below this line
# ================================================================

class TestGraph(unittest.TestCase):

    def setUp(self):
        self.g = Digraph()
        self.na = Node('a')
        self.nb = Node('b')
        self.nc = Node('c')
        self.g.add_node(self.na)
        self.g.add_node(self.nb)
        self.g.add_node(self.nc)
        self.e1 = WeightedEdge(self.na, self.nb, 15, 10)
        self.e2 = WeightedEdge(self.na, self.nc, 14, 6)
        self.e3 = WeightedEdge(self.nb, self.nc, 3, 1)
        self.g.add_edge(self.e1)
        self.g.add_edge(self.e2)
        self.g.add_edge(self.e3)

    def test_weighted_edge_str(self):
        self.assertEqual(str(self.e1), "a->b (15, 10)")
        self.assertEqual(str(self.e2), "a->c (14, 6)")
        self.assertEqual(str(self.e3), "b->c (3, 1)")

    def test_weighted_edge_total_distance(self):
        self.assertEqual(self.e1.get_total_distance(), 15)
        self.assertEqual(self.e2.get_total_distance(), 14)
        self.assertEqual(self.e3.get_total_distance(), 3)

    def test_weighted_edge_outdoor_distance(self):
        self.assertEqual(self.e1.get_outdoor_distance(), 10)
        self.assertEqual(self.e2.get_outdoor_distance(), 6)
        self.assertEqual(self.e3.get_outdoor_distance(), 1)

    def test_add_edge_to_nonexistent_node_raises(self):
        node_not_in_graph = Node('q')
        no_src = WeightedEdge(self.nb, node_not_in_graph, 5, 5)
        no_dest = WeightedEdge(node_not_in_graph, self.na, 5, 5)

        with self.assertRaises(ValueError):
            self.g.add_edge(no_src)
        with self.assertRaises(ValueError):
            self.g.add_edge(no_dest)

    def test_add_existing_node_raises(self):
        with self.assertRaises(ValueError):
            self.g.add_node(self.na)

    def test_graph_str(self):
        expected = "a->b (15, 10)\na->c (14, 6)\nb->c (3, 1)"
        self.assertEqual(str(self.g), expected)


    if __name__ == '__main__':
      unittest.main(argv=['first-arg-is-ignored'], exit=False)





----------------------------------------------------------------------
Ran 0 tests in 0.000s

OK


In [2]:
%cd

/root


In [38]:


# Problem 2b: Implementing load_map
def load_map(map_filename):
    """
    Parses the map file and constructs a directed graph

    Parameters:
        map_filename : name of the map file

    Assumes:
        Each entry in the map file consists of the following four positive
        integers, separated by a blank space:
            From To TotalDistance DistanceOutdoors
        e.g.
            32 76 54 23
        This entry would become an edge from 32 to 76.

    Returns:
        a Digraph representing the map
    """

    # TODO
    # read in map_file example text "32 36 70 0"
    f = open(map_filename, 'r', encoding='utf-8')
    # create the mit diagraph
    nodes = [] 
    edges = []
    mit = Digraph()
    # loop through each line
    for line in f:
        # grab the src, destination, total, and outdoor distances
        (src, dest, total_distance, outdoor_distance) = line.split(" ")
        src, dest, total_distance, outdoor_distance = str(src), str(dest), int(total_distance), int(outdoor_distance)
        print(src, dest, total_distance, outdoor_distance)
        nodes.append(Node(src))
        edges.append(WeightedEdge(src, dest, total_distance, outdoor_distance))
    for node in nodes:
      try:
        # add it to the mit diagraph
        mit.add_node(node)
      except ValueError as error:
        print(error)
        continue
    for edge in edges:
      try:
        mit.add_edge(edge)
      except ValueError as error:
        print(error)
        continue

    print("Loading map from file...")
    
load_map('mit_map.txt')


32 36 70 0
36 32 70 0
32 57 30 0
57 32 30 0
32 76 80 50
76 32 80 50
32 68 110 80
68 32 110 80
76 68 72 30
68 76 72 30
68 66 51 0
66 68 51 0
68 56 80 70
56 68 80 70
66 56 40 0
56 66 40 0
76 66 130 100
66 76 130 100
56 18 35 0
18 56 35 0
56 16 30 0
16 56 30 0
36 26 34 0
26 36 34 0
24 13 35 30
13 24 35 30
36 34 25 0
34 36 25 0
32 16 100 50
16 32 100 50
32 12 100 80
12 32 100 80
26 16 45 0
16 26 45 0
26 12 30 25
12 26 30 25
26 24 25 20
24 26 25 20
16 8 25 0
8 16 25 0
34 24 27 0
24 34 27 0
24 12 33 0
12 24 33 0
12 4 56 0
4 12 56 0
8 6 39 0
6 8 39 0
39 37 32 0
37 39 32 0
39 13 70 50
13 39 70 50
31 13 30 25
13 31 30 25
8 4 47 0
4 8 47 0
6 2 41 0
2 6 41 0
2 14 51 0
14 2 51 0
2 4 36 0
4 2 36 0
14 50 50 23
50 14 50 23
4 10 47 0
10 4 47 0
2 10 70 50
10 2 70 50
10 3 32 0
3 10 32 0
4 3 60 50
3 4 60 50
2 3 70 50
3 2 70 50
2 1 75 60
1 2 75 60
4 1 80 65
1 4 80 65
3 1 36 0
1 3 36 0
5 1 32 0
1 5 32 0
7 5 20 0
5 7 20 0
7 3 25 0
3 7 25 0
13 10 30 0
10 13 30 0
9 13 40 0
13 9 40 0
9 7 20 0
7 9 20 0
34 38 25

AttributeError: ignored

In [17]:
# 6.0002 Problem Set 5
# Graph optimization
# Name:
# Collaborators:
# Time:

#
# Finding shortest paths through MIT buildings
#
import unittest


#
# Problem 2: Building up the Campus Map
#
# Problem 2a: Designing your graph
#
# What do the graph's nodes represent in this problem? What
# do the graph's edges represent? Where are the distances
# represented?
#
# Answer:
#


# Problem 2b: Implementing load_map
def load_map(map_filename):
    """
    Parses the map file and constructs a directed graph

    Parameters:
        map_filename : name of the map file

    Assumes:
        Each entry in the map file consists of the following four positive
        integers, separated by a blank space:
            From To TotalDistance DistanceOutdoors
        e.g.
            32 76 54 23
        This entry would become an edge from 32 to 76.

    Returns:
        a Digraph representing the map
    """

    # TODO
    # read in map_file example text "32 36 70 0"
    f = open(map_filename, 'r', encoding='utf-8')
    # create the mit diagraph 
    mit = Digraph()
    # loop through each line
    for line in f:
      # grab the src, destination, total, and outdoor distances
      (src, dest, total_distance, outdoor_distance) = line.split(" ")
      src, dest, total_distance, outdoor_distance = src, dest, int(total_distance), int(outdoor_distance)
      print(src, dest, total_distance, outdoor_distance)
      # create the node
      n = Node(f"{src}")
      # add it to the mit diagraph
      mit.add_node(n)
      # create the weighted edge
      we = WeightedEdge(src, dest, total_distance, outdoor_distance)
      mit.add_edge(we)

    return mit
    print("Loading map from file...")
load_map(mit_map.txt')
# Problem 2c: Testing load_map
# Include the lines used to test load_map below, but comment them out


#
# Problem 3: Finding the Shorest Path using Optimized Search Method
#
# Problem 3a: Objective function
#
# What is the objective function for this problem? What are the constraints?
#
# Answer:
#

# Problem 3b: Implement get_best_path
def get_best_path(digraph, start, end, path, max_dist_outdoors, best_dist,
                  best_path):
    """
    Finds the shortest path between buildings subject to constraints.

    Parameters:
        digraph: Digraph instance
            The graph on which to carry out the search
        start: string
            Building number at which to start
        end: string
            Building number at which to end
        path: list composed of [[list of strings], int, int]
            Represents the current path of nodes being traversed. Contains
            a list of node names, total distance traveled, and total
            distance outdoors.
        max_dist_outdoors: int
            Maximum distance spent outdoors on a path
        best_dist: int
            The smallest distance between the original start and end node
            for the initial problem that you are trying to solve
        best_path: list of strings
            The shortest path found so far between the original start
            and end node.

    Returns:
        A tuple with the shortest-path from start to end, represented by
        a list of building numbers (in strings), [n_1, n_2, ..., n_k],
        where there exists an edge from n_i to n_(i+1) in digraph,
        for all 1 <= i < k and the distance of that path.

        If there exists no path that satisfies max_total_dist and
        max_dist_outdoors constraints, then return None.
    """
    # TODO
    pass


# Problem 3c: Implement directed_dfs
def directed_dfs(digraph, start, end, max_total_dist, max_dist_outdoors):
    """
    Finds the shortest path from start to end using a directed depth-first
    search. The total distance traveled on the path must not
    exceed max_total_dist, and the distance spent outdoors on this path must
    not exceed max_dist_outdoors.

    Parameters:
        digraph: Digraph instance
            The graph on which to carry out the search
        start: string
            Building number at which to start
        end: string
            Building number at which to end
        max_total_dist: int
            Maximum total distance on a path
        max_dist_outdoors: int
            Maximum distance spent outdoors on a path

    Returns:
        The shortest-path from start to end, represented by
        a list of building numbers (in strings), [n_1, n_2, ..., n_k],
        where there exists an edge from n_i to n_(i+1) in digraph,
        for all 1 <= i < k

        If there exists no path that satisfies max_total_dist and
        max_dist_outdoors constraints, then raises a ValueError.
    """
    # TODO
    pass

32 36 70 0


AttributeError: ignored

In [23]:
# ================================================================
# Begin tests -- you do not need to modify anything below this line
# ================================================================

class Ps2Test(unittest.TestCase):
    LARGE_DIST = 99999

    def setUp(self):
        self.graph = load_map("mit_map.txt")

    def test_load_map_basic(self):
        self.assertTrue(isinstance(self.graph, Digraph))
        self.assertEqual(len(self.graph.nodes), 37)
        all_edges = []
        for _, edges in self.graph.edges.items():
            all_edges += edges  # edges must be dict of node -> list of edges
        all_edges = set(all_edges)
        self.assertEqual(len(all_edges), 129)

    # def _print_path_description(self, start, end, total_dist, outdoor_dist):
    #     constraint = ""
    #     if outdoor_dist != Ps2Test.LARGE_DIST:
    #         constraint = "without walking more than {}m outdoors".format(
    #             outdoor_dist)
    #     if total_dist != Ps2Test.LARGE_DIST:
    #         if constraint:
    #             constraint += ' or {}m total'.format(total_dist)
    #         else:
    #             constraint = "without walking more than {}m total".format(
    #                 total_dist)

    #     print("------------------------")
    #     print("Shortest path from Building {} to {} {}".format(
    #         start, end, constraint))

    # def _test_path(self,
    #                expectedPath,
    #                total_dist=LARGE_DIST,
    #                outdoor_dist=LARGE_DIST):
    #     start, end = expectedPath[0], expectedPath[-1]
    #     self._print_path_description(start, end, total_dist, outdoor_dist)
    #     dfsPath = directed_dfs(self.graph, start, end, total_dist, outdoor_dist)
    #     print("Expected: ", expectedPath)
    #     print("DFS: ", dfsPath)
    #     self.assertEqual(expectedPath, dfsPath)

    # def _test_impossible_path(self,
    #                           start,
    #                           end,
    #                           total_dist=LARGE_DIST,
    #                           outdoor_dist=LARGE_DIST):
    #     self._print_path_description(start, end, total_dist, outdoor_dist)
    #     with self.assertRaises(ValueError):
    #         directed_dfs(self.graph, start, end, total_dist, outdoor_dist)

    # def test_path_one_step(self):
    #     self._test_path(expectedPath=['32', '56'])

    # def test_path_no_outdoors(self):
    #     self._test_path(
    #         expectedPath=['32', '36', '26', '16', '56'], outdoor_dist=0)

    # def test_path_multi_step(self):
    #     self._test_path(expectedPath=['2', '3', '7', '9'])

    # def test_path_multi_step_no_outdoors(self):
    #     self._test_path(
    #         expectedPath=['2', '4', '10', '13', '9'], outdoor_dist=0)

    # def test_path_multi_step2(self):
    #     self._test_path(expectedPath=['1', '4', '12', '32'])

    # def test_path_multi_step_no_outdoors2(self):
    #     self._test_path(
    #         expectedPath=['1', '3', '10', '4', '12', '24', '34', '36', '32'],
    #         outdoor_dist=0)

    # def test_impossible_path1(self):
    #     self._test_impossible_path('8', '50', outdoor_dist=0)

    # def test_impossible_path2(self):
    #     self._test_impossible_path('10', '32', total_dist=100)


if __name__ == '__main__':
  unittest.main(argv=['first-arg-is-ignored'], exit=False)


F......

32 36 70 0
36 32 70 0
32 57 30 0
Duplicate node
57 32 30 0
32 76 80 50
Duplicate node
76 32 80 50
32 68 110 80
Duplicate node
68 32 110 80
76 68 72 30
Duplicate node
68 76 72 30
Duplicate node
68 66 51 0
Duplicate node
66 68 51 0
68 56 80 70
Duplicate node
56 68 80 70
66 56 40 0
Duplicate node
56 66 40 0
Duplicate node
76 66 130 100
Duplicate node
66 76 130 100
Duplicate node
56 18 35 0
Duplicate node
18 56 35 0
56 16 30 0
Duplicate node
16 56 30 0
36 26 34 0
Duplicate node
26 36 34 0
24 13 35 30
13 24 35 30
36 34 25 0
Duplicate node
34 36 25 0
32 16 100 50
Duplicate node
16 32 100 50
Duplicate node
32 12 100 80
Duplicate node
12 32 100 80
26 16 45 0
Duplicate node
16 26 45 0
Duplicate node
26 12 30 25
Duplicate node
12 26 30 25
Duplicate node
26 24 25 20
Duplicate node
24 26 25 20
Duplicate node
16 8 25 0
Duplicate node
8 16 25 0
34 24 27 0
Duplicate node
24 34 27 0
Duplicate node
24 12 33 0
Duplicate node
12 24 33 0
Duplicate node
12 4 56 0
Duplicate node
4 12 56 0
8 6 39 0
Duplicate


FAIL: test_load_map_basic (__main__.Ps2Test)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "<ipython-input-23-616a19022e3d>", line 13, in test_load_map_basic
    self.assertEqual(len(self.graph.nodes), 37)
AssertionError: 0 != 37

----------------------------------------------------------------------
Ran 7 tests in 0.127s

FAILED (failures=1)
